In [17]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import re
import time
import pandas as pd

In [18]:
# DeprecationWarning: executable_path has been deprecated, please pass in a Service object
driver_path = '/usr/bin/chromedriver'

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')

# Headless run option for later use
# chrome_options.add_argument('--headless')

# Media query = desktop 
options.add_argument('--window-size=1600,900')

# Fixes 'data:,' url issue 
options.add_argument('--remote-debugging-port=9225')

driver = webdriver.Chrome(executable_path=driver_path, options=options)

/tmp/ipykernel_9388/2031286696.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path, options=options)


In [19]:
# Open website URL
url = 'https://korean.visitkorea.or.kr/main/main.do#home'
driver.get(url)

# Page loading buffer
time.sleep(2)

# Ask search keyword
# search_keyword = input('Enter search keyword: ')
search_keyword = "서울"

# Ask file name and path
# save_file = input('Enter full CSV file path and name: ')
file_path = "/home/linux-user/Downloads/dataset.csv"
log_path = "/home/linux-user/Downloads/log.txt"

# Locate search button
# Use attributes instead of 'driver.find_element_by_class_name' method
driver.find_element(By.CSS_SELECTOR, '.btn_search').click()

# Locate search box
# 'element = driver.find_element(By.CSS_SELECTOR, '#inp_search').click()' returns none, seperate the action
element = driver.find_element(By.CSS_SELECTOR, '#inp_search')
element.click()

# Send query keyword to search box
# 'element.sendkeys(search_keyword)' is deprecated
element.send_keys(search_keyword)

# Perform search 
driver.find_element(By.CSS_SELECTOR, ".btn_search").click()

# Page loading buffer until delayed popup appears
time.sleep(15)

# Disable fullscreen popup
driver.find_element(By.CSS_SELECTOR, "._sa_t-size__medium").click()
time.sleep(2)

In [20]:
# Prepare to save log as file
log = sys.stdout
f = open(log_path, 'a', encoding='UTF-8')
sys.stdout = f

# Announce the process
now = datetime.now()
current_datetime = now.strftime("%Y-%m-%d %H:%M:%S")
print("The script is activated.")
print("Current time: {}".format(current_datetime))

# Initiate bs4 html.parser to read the whole HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Exclude ad banner area from first page
unwanted = soup.select_one('li.banner_area')
unwanted.extract()

# Collect search results
search_result = soup.find('ul', class_='list_thumType')

# Define data arrays
title_list = []
region_list = []
hashtag_list = []

# Check the last page
current_page = 1
page_box = driver.find_element(By.ID, "page_box")
pages = page_box.find_elements(By.TAG_NAME, "a")
last_page = int(pages[-1].get_attribute('id'))

In [21]:
# Start page turner
try:
    while current_page < last_page:
        if current_page == 1:
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            # Exclude ad banner area from first page
            unwanted = soup.select_one('li.banner_area')
            unwanted.extract()  
            search_result = soup.find('ul', class_='list_thumType')
            for i in search_result:
                title = i.select_one("div.tit").get_text()
                title_list.append(title)
                try:
                    region = i.select_one("p").get_text()
                    region_list.append(region)                    
                except AttributeError:
                    region = "N/A"
                    region_list.append(region)
                try:    
                    hashtag = i.select_one("p.tag_type").get_text()
                    hashtag_list.append(hashtag)                    
                except AttributeError:
                    hashtag = "N/A"
                    hashtag_list.append(hashtag)
            current_page += 1
            current_page_str = str(current_page)
            next_page = driver.find_element(By.LINK_TEXT, current_page_str)
            next_page.send_keys(Keys.ENTER)
            time.sleep(7)
        elif current_page % 5 != 0:
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            search_result = soup.find('ul', class_='list_thumType')
            for i in search_result:
                title = i.select_one("div.tit").get_text()
                title_list.append(title)
                try:
                    region = i.select_one("p").get_text()
                    region_list.append(region)                    
                except AttributeError:
                    region = "N/A"
                    region_list.append(region)
                try:    
                    hashtag = i.select_one("p.tag_type").get_text()
                    hashtag_list.append(hashtag)                    
                except AttributeError:
                    hashtag = "N/A"
                    hashtag_list.append(hashtag)
            current_page += 1
            current_page_str = str(current_page)
            next_page = driver.find_element(By.LINK_TEXT, current_page_str)
            next_page.send_keys(Keys.ENTER)
            time.sleep(7)        
        elif current_page % 5 == 0:
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            search_result = soup.find('ul', class_='list_thumType')
            for i in search_result:
                title = i.select_one("div.tit").get_text()
                title_list.append(title)
                try:
                    region = i.select_one("p").get_text()
                    region_list.append(region)                    
                except AttributeError:
                    region = "N/A"
                    region_list.append(region)
                try:    
                    hashtag = i.select_one("p.tag_type").get_text()
                    hashtag_list.append(hashtag)                    
                except AttributeError:
                    hashtag = "N/A"
                    hashtag_list.append(hashtag)
            current_page += 1
            current_page_str = str(current_page)
            next_page = driver.find_element(By.CSS_SELECTOR, ".btn_next")
            next_page.send_keys(Keys.ENTER)
            time.sleep(7)
        elif current_page == last_page:
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            search_result = soup.find('ul', class_='list_thumType')
            for i in search_result:
                title = i.select_one("div.tit").get_text()
                title_list.append(title)
                try:
                    region = i.select_one("p").get_text()
                    region_list.append(region)                    
                except AttributeError:
                    region = "N/A"
                    region_list.append(region)
                try:    
                    hashtag = i.select_one("p.tag_type").get_text()
                    hashtag_list.append(hashtag)                    
                except AttributeError:
                    hashtag = "N/A"
                    hashtag_list.append(hashtag)
            break

except Exception as e:
    now = datetime.now()
    current_datetime = now.strftime("%Y-%m-%d %H:%M:%S")
    print("An error has occured. ", e)
    print("Current time: {}".format(current_datetime))

finally:
    # Initiate Pandas dataframe
    dataset = pd.DataFrame()
    dataset['Title'] = title_list
    dataset['Region'] = region_list
    dataset['Hashtags'] = hashtag_list
    dataset['Hashtags'].df.replace('#', ' ', inplace=True)

    # Save dataset as CSV file
    dataset.to_csv(file_path, encoding="utf-8-sig", index=False)

    now = datetime.now()
    current_datetime = now.strftime("%Y-%m-%d %H:%M:%S")
    print("The script has been finished successfully.")
    print("Current time: {}".format(current_datetime))

    # Save log as TXT file
    sys.stdout = log
    f.close()